In [17]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [20]:
# parameters

learning_rate = 0.001
train_epochs = 15
batch_size = 100

In [21]:
# MNIST dataset

mnist_train = dsets.MNIST(root = 'MNIST_data/',
                         train = True,
                         transform = transforms.ToTensor(),
                         download = True)

mnist_test = dsets.MNIST(root = 'MNIST_data/',
                        train =False,
                        transform = transforms.ToTensor(),
                        download = True)

In [22]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                         batch_size = batch_size,
                                         shuffle = True,
                                         drop_last = True)

In [24]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc = nn.Linear(7*7*64, 10, bias = True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [25]:
model = CNN().to(device)

In [26]:
model

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3136, out_features=10, bias=True)
)

In [28]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [30]:
# training

total_batch = len(data_loader)
print('Learning Started.')

for epoch in range(train_epochs):
    avg_cost = 0
    
    for X, y in data_loader:
        X = X.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = criterion(hypothesis, y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print('Epochs : {} Cost = {}'.format(epoch+1, avg_cost))
    
print('Learning Finished.')

Learning Started.
Epochs : 1 Cost = 0.22384990751743317
Epochs : 2 Cost = 0.06228359788656235
Epochs : 3 Cost = 0.04500725865364075
Epochs : 4 Cost = 0.035720840096473694
Epochs : 5 Cost = 0.02907080389559269
Epochs : 6 Cost = 0.02495693415403366
Epochs : 7 Cost = 0.020894620567560196
Epochs : 8 Cost = 0.018191592767834663
Epochs : 9 Cost = 0.014966728165745735
Epochs : 10 Cost = 0.01227874867618084
Epochs : 11 Cost = 0.010512229055166245
Epochs : 12 Cost = 0.009993219748139381
Epochs : 13 Cost = 0.008786349557340145
Epochs : 14 Cost = 0.007422414608299732
Epochs : 15 Cost = 0.005664475727826357
Learning Finished.


In [31]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy : ', accuracy.item())

C:\Users\seho\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\seho\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy :  0.9864000082015991


In [32]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2))
        
        self.fc1 = nn.Linear(3*3*128, 625, bias = True)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias = True)
        
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [33]:
model = CNN().to(device)
print(model)

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=1152, out_features=625, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=625, out_features=10, bias=True)
)


In [34]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [35]:
# training

total_batch = len(data_loader)
print('Learning Started.')

for epoch in range(train_epochs):
    avg_cost = 0
    
    for X, y in data_loader:
        X = X.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = criterion(hypothesis, y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print('Epochs : {} Cost = {}'.format(epoch+1, avg_cost))
    
print('Learning Finished.')

Learning Started.
Epochs : 1 Cost = 0.16946133971214294
Epochs : 2 Cost = 0.04464116320014
Epochs : 3 Cost = 0.030296185985207558
Epochs : 4 Cost = 0.02213365212082863
Epochs : 5 Cost = 0.017461048439145088
Epochs : 6 Cost = 0.014763814397156239
Epochs : 7 Cost = 0.012204213067889214
Epochs : 8 Cost = 0.010149491019546986
Epochs : 9 Cost = 0.010854247957468033
Epochs : 10 Cost = 0.00828423723578453
Epochs : 11 Cost = 0.008074430748820305
Epochs : 12 Cost = 0.0060691554099321365
Epochs : 13 Cost = 0.006597358267754316
Epochs : 14 Cost = 0.00590180978178978
Epochs : 15 Cost = 0.004968435037881136
Learning Finished.


In [36]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy : ', accuracy.item())

Accuracy :  0.980400025844574
